In [18]:
!pip install ortools
from ortools.linear_solver import pywraplp
import json
import requests

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/barrios.json")
ntas_db = json.loads(response.text)

response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)

In [19]:
for neighbor in ntas_db:
    neighbor["vehicles_needed_per_person"] = {
        "engine": 0,
        "ladder": 0,
        "rescue": 0,
        "squad": 0,
        "hazardous": 0
    }

In [20]:
ntas_db

[{'boro_name': 'Queens',
  'name': 'St. Albans',
  'shape_area': 77412747.847,
  'nta': 'QN08',
  'population': 48593,
  'vehicles_needed_per_person': {'engine': 0,
   'ladder': 0,
   'rescue': 0,
   'squad': 0,
   'hazardous': 0}},
 {'boro_name': 'Bronx',
  'name': 'Van Cortlandt Village',
  'shape_area': 25666124.5948,
  'nta': 'BX28',
  'population': 50100,
  'vehicles_needed_per_person': {'engine': 0,
   'ladder': 0,
   'rescue': 0,
   'squad': 0,
   'hazardous': 0}},
 {'boro_name': 'Queens',
  'name': 'South Ozone Park',
  'shape_area': 82461393.7368,
  'nta': 'QN55',
  'population': 75878,
  'vehicles_needed_per_person': {'engine': 0,
   'ladder': 0,
   'rescue': 0,
   'squad': 0,
   'hazardous': 0}},
 {'boro_name': 'Brooklyn',
  'name': 'Canarsie',
  'shape_area': 82089678.6389,
  'nta': 'BK50',
  'population': 83693,
  'vehicles_needed_per_person': {'engine': 0,
   'ladder': 0,
   'rescue': 0,
   'squad': 0,
   'hazardous': 0}},
 {'boro_name': 'Bronx',
  'name': 'Mount Hope',
 

In [15]:
incidents_db

[{'nta': 'MN03',
  'units': ['ladder'],
  'incident_duration': 779.0,
  'is_first_shift': True},
 {'nta': 'MN23',
  'units': ['engine'],
  'incident_duration': 1935.0,
  'is_first_shift': True},
 {'nta': 'BX09',
  'units': ['ladder', 'engine', 'ladder', 'ladder'],
  'incident_duration': 1127.0,
  'is_first_shift': False},
 {'nta': 'BK43',
  'units': ['ladder'],
  'incident_duration': 521.0,
  'is_first_shift': True},
 {'nta': 'MN01',
  'units': ['ladder'],
  'incident_duration': 364.0,
  'is_first_shift': False},
 {'nta': 'MN20',
  'units': ['ladder'],
  'incident_duration': 1397.0,
  'is_first_shift': False},
 {'nta': 'MN03',
  'units': ['engine'],
  'incident_duration': 861.0,
  'is_first_shift': False},
 {'nta': 'BX26',
  'units': ['engine'],
  'incident_duration': 1924.0,
  'is_first_shift': False},
 {'nta': 'BX14',
  'units': ['engine'],
  'incident_duration': 1206.0,
  'is_first_shift': True},
 {'nta': 'QN55',
  'units': ['ladder', 'ladder', 'ladder', 'ladder', 'rescue', 'engine'

In [21]:
for neighbor in ntas_db:
    for incident in incidents_db:
        if incident["nta"] == neighbor["nta"]:
            neighbor["vehicles_needed_per_person"]["engine"] += incident["units"].count("engine")
            neighbor["vehicles_needed_per_person"]["ladder"] += incident["units"].count("ladder")
            neighbor["vehicles_needed_per_person"]["rescue"] += incident["units"].count("rescue")
            neighbor["vehicles_needed_per_person"]["squad"] += incident["units"].count("squad")
            neighbor["vehicles_needed_per_person"]["hazardous"] += incident["units"].count("hazardous")

In [22]:
ntas_db

[{'boro_name': 'Queens',
  'name': 'St. Albans',
  'shape_area': 77412747.847,
  'nta': 'QN08',
  'population': 48593,
  'vehicles_needed_per_person': {'engine': 1812,
   'ladder': 1987,
   'rescue': 109,
   'squad': 53,
   'hazardous': 94}},
 {'boro_name': 'Bronx',
  'name': 'Van Cortlandt Village',
  'shape_area': 25666124.5948,
  'nta': 'BX28',
  'population': 50100,
  'vehicles_needed_per_person': {'engine': 0,
   'ladder': 0,
   'rescue': 0,
   'squad': 0,
   'hazardous': 0}},
 {'boro_name': 'Queens',
  'name': 'South Ozone Park',
  'shape_area': 82461393.7368,
  'nta': 'QN55',
  'population': 75878,
  'vehicles_needed_per_person': {'engine': 2811,
   'ladder': 3063,
   'rescue': 218,
   'squad': 103,
   'hazardous': 140}},
 {'boro_name': 'Brooklyn',
  'name': 'Canarsie',
  'shape_area': 82089678.6389,
  'nta': 'BK50',
  'population': 83693,
  'vehicles_needed_per_person': {'engine': 0,
   'ladder': 0,
   'rescue': 0,
   'squad': 0,
   'hazardous': 0}},
 {'boro_name': 'Bronx',
  '

In [25]:
for neighbor in ntas_db:
    vehicles_needed = neighbor["vehicles_needed_per_person"]
    if neighbor["population"] != 0:
        vehicles_needed["engine"] = vehicles_needed["engine"] / neighbor["population"]
        vehicles_needed["ladder"] = vehicles_needed["ladder"] / neighbor["population"]
        vehicles_needed["rescue"] = vehicles_needed["rescue"] / neighbor["population"]
        vehicles_needed["squad"] = vehicles_needed["squad"] / neighbor["population"]
        vehicles_needed["hazardous"] = vehicles_needed["hazardous"] / neighbor["population"]

In [26]:
ntas_db

[{'boro_name': 'Queens',
  'name': 'St. Albans',
  'shape_area': 77412747.847,
  'nta': 'QN08',
  'population': 48593,
  'vehicles_needed_per_person': {'engine': 1.5791997158407295e-11,
   'ladder': 1.7317162446884818e-11,
   'rescue': 9.499600939660018e-13,
   'squad': 4.619072016531935e-13,
   'hazardous': 8.192316406679281e-13}},
 {'boro_name': 'Bronx',
  'name': 'Van Cortlandt Village',
  'shape_area': 25666124.5948,
  'nta': 'BX28',
  'population': 50100,
  'vehicles_needed_per_person': {'engine': 0.0,
   'ladder': 0.0,
   'rescue': 0.0,
   'squad': 0.0,
   'hazardous': 0.0}},
 {'boro_name': 'Queens',
  'name': 'South Ozone Park',
  'shape_area': 82461393.7368,
  'nta': 'QN55',
  'population': 75878,
  'vehicles_needed_per_person': {'engine': 6.4344765141051685e-12,
   'ladder': 7.011313256031353e-12,
   'rescue': 4.990095624599526e-13,
   'squad': 2.357705730888767e-13,
   'hazardous': 3.2046485662565764e-13}},
 {'boro_name': 'Brooklyn',
  'name': 'Canarsie',
  'shape_area': 8208